In [1]:
import jax
import jax.numpy as jnp
from jax import random, grad, jit, vmap
from jax.random import split, PRNGKey
def init_RWKV_params(input_size, emb_size, h_size,  num_layer, out_size, Ny, Nx, key):
    key, emb_key, init_x_key, init_y_key, last_x1_key, last_x2_key, last_y1s_key, last_y1e_key, last_y2_key, key_tout, key_txout, key_talpha_out, key_tbeta_out, key_tlast_x, key_clast_x, key_cxout = split(key, 16)
    wemb = random.uniform(emb_key, (input_size, emb_size), minval=-1e-4, maxval=1e-4) #input hasn't been connected
    x_init = random.uniform(init_x_key, (Nx, emb_size), minval=-1e-4, maxval=1e-4)
    y_init = random.uniform(init_y_key, (Ny, emb_size), minval=-1e-4, maxval=1e-4)
    last_x1_init = random.uniform(last_x1_key, (Nx, num_layer, emb_size), minval=-1e-4, maxval=1e-4)
    last_x2_init = random.uniform(last_x2_key, (Nx, num_layer, emb_size), minval=-1e-4, maxval=1e-4)
    last_y1s_init = random.uniform(last_y1s_key, (Ny+1, num_layer, emb_size), minval=-1e-4, maxval=1e-4)
    last_y1e_init = random.uniform(last_y1e_key, (Ny+1, num_layer, emb_size), minval=-1e-4, maxval=1e-4)
    last_y2_init = random.uniform(last_y2_key, (Ny, num_layer, emb_size), minval=-1e-4, maxval=1e-4)
    t_alpha_init_x, t_alpha_init_y, t_beta_init_x, t_beta_init_y = jnp.zeros((Nx, num_layer, h_size)), jnp.zeros((Ny, num_layer, h_size)), jnp.zeros((Nx, num_layer, h_size)), jnp.zeros((Ny, num_layer, h_size))
    t_xout = random.normal(key_tout, (Ny, Nx, num_layer, emb_size, 2*emb_size))
    t_alphaout = random.normal(key_talpha_out, (Ny, Nx, num_layer, h_size, 2*h_size))
    t_betaout = random.normal(key_tbeta_out, (Ny, Nx, num_layer, h_size, 2*h_size))
    c_xout = random.normal(key_cxout, (Ny, Nx, num_layer, emb_size, 2*emb_size))    
    emb_size, h_size = 2*emb_size, 2*h_size #tensor product the input from two directions and concantenate the hidden state
    wln_in, bln_in, wln_out, bln_out = jnp.ones((Ny, Nx, emb_size)), jnp.zeros((Ny, Nx, emb_size)), jnp.ones((Ny, Nx, emb_size)), jnp.zeros((Ny, Nx, emb_size))  #in&out layer_norm params
    wln, bln = jnp.ones((2, Ny, Nx, num_layer, emb_size)), jnp.zeros((2, Ny, Nx, num_layer, emb_size))  #time&channel layer_norm params

    # time mixing params
    decay = jnp.tile(-5 + jnp.array([8*(jnp.arange(h_size)/(h_size-1))**(0.7 + 1.3*i/(num_layer-1)) for i in range(num_layer)]), (Ny, Nx, 1, 1))
    bonus = jnp.tile(0.5*(jnp.arange(h_size)%3-1)+jnp.log(0.3), (Ny, Nx, num_layer, 1))
    t_mix_k = jnp.tile(jnp.array([(jnp.arange(emb_size) / emb_size) ** (1 - i / num_layer) for i in range(num_layer)]), (Ny, Nx, 1, 1))
    t_mix_v = t_mix_k + jnp.transpose(jnp.tile(jnp.arange(num_layer) * 0.3 / (num_layer - 1), (Ny, Nx, emb_size, 1)), (0, 1, 3, 2))
    t_mix_r = 0.5 * t_mix_k
    t_wk, t_wv, t_wr = jnp.zeros((Ny, Nx, num_layer, h_size, emb_size)), jnp.zeros((Ny, Nx, num_layer, h_size, emb_size)), jnp.zeros((Ny, Nx, num_layer, h_size, emb_size))
    t_wout = jnp.sqrt(h_size/emb_size)*random.normal(key_tout, (Ny, Nx, num_layer, emb_size, h_size))
    t_wlast_x = random.normal(key_tlast_x, (Ny, Nx, num_layer, emb_size, 2*emb_size)) #since last_x is twice larger than x

    # channel mixing params
    c_mix_k, c_mix_r = t_mix_k, t_mix_k
    c_wr, c_wv, c_wk = jnp.zeros((Ny, Nx, num_layer, emb_size, emb_size)), jnp.zeros((Ny, Nx, num_layer, emb_size, emb_size)), jnp.zeros((Ny, Nx, num_layer, emb_size, emb_size))
    c_wlast_x = random.normal(key_clast_x, (Ny, Nx, num_layer, emb_size, 2*emb_size)) #since last_x is twice larger than x
    # output params
    whead, bhead = jnp.tile(jnp.eye(emb_size), (Ny, Nx, 1, 1)), jnp.zeros((Ny, Nx, emb_size))
    wprob, bprob, wphase, bphase = jnp.zeros((out_size, emb_size)), jnp.zeros((out_size)), jnp.zeros((out_size, emb_size)), jnp.zeros((out_size))
    RWKV_cell_params = wln[0], bln[0], wln[1], bln[1], decay, bonus, t_mix_r, t_mix_v, t_mix_k, t_wr, t_wv, t_wk, t_wout, t_wlast_x, c_mix_k, c_mix_r, c_wr, c_wv, c_wk, c_wlast_x, t_xout, t_alphaout, t_betaout, c_xout

    return (wemb, x_init, y_init, last_x1_init, last_x2_init, last_y1s_init, last_y1e_init, last_y2_init, t_alpha_init_x, t_alpha_init_y, t_beta_init_x, t_beta_init_y,  wln_in, bln_in, wln_out, bln_out, whead, bhead, wprob, bprob, wphase, bphase, RWKV_cell_params)

In [2]:
def int_to_binary_array(x, num_bits):
    """
    Converts an array of integers to their binary representation arrays with a fixed number of bits.
    This function is designed to be compatible with Jax's vmap for vectorization over an array of integers.

    Parameters:
    - x: An array of integers, the numbers to convert.
    - num_bits: Integer, the fixed number of bits for the binary representation.

    Returns:
    - A 2D Jax array where each row is the binary representation of an integer in 'x'.
    """
    # Create an array of bit positions: [2^(num_bits-1), 2^(num_bits-2), ..., 1]
    powers_of_two = 2 ** jnp.arange(num_bits - 1, -1, -1)

    # Expand dims of x and powers_of_two for broadcasting
    x_expanded = x[:, None]
    powers_of_two_expanded = powers_of_two[None, :]

    # Perform bitwise AND between each number and each power of two, then right shift to get the bit value
    binary_matrix = (x_expanded & powers_of_two_expanded) >> jnp.arange(num_bits - 1, -1, -1)

    return binary_matrix.astype(jnp.int32)  # Ensure the result is integer


def binary_array_to_int(binary_array, num_bits):
    """
    Converts a 2D array of binary representations to their decimal equivalents.

    Parameters:
    - binary_array: A 2D Jax array where each row represents a binary number.

    Returns:
    - A 1D Jax array of integers, the decimal equivalents of the binary representations.
    """
    powers_of_two = 2 ** jnp.arange(num_bits - 1, -1, -1)
    # Multiply each bit by its corresponding power of two and sum the results
    decimals = jnp.dot(binary_array, powers_of_two)
    return decimals

In [13]:
from functools import partial
from jax.lax import scan
from jax.random import categorical

@partial(jax.jit, static_argnames=['fixed_params'])
def sample_prob(params, fixed_params, ny_nx_indices, key):

    Ny, Nx, py, px, mag_fixed, magnetization, num_layer = fixed_params
    int_to_binary = partial(int_to_binary_array, num_bits=px * py)
    
    def scan_fun_1d(carry_1d, indices):
        ny, nx = indices
        
        input_x, input_yi, last_x1, last_x1s, last_x1e, last_x2, last_y1i, last_y2i, t_alpha_state_x1, t_beta_state_x1, t_alpha_state_yi, t_beta_state_yi, key = carry_1d
    
        rnn_input = jnp.concatenate((input_yi, input_x[nx]), axis=0)
        last_state = jnp.concatenate((last_y2i, last_x1s[nx], last_x2[nx], last_x1e[nx]), axis=1)
        t_alpha_state = jnp.concatenate((t_alpha_state_yi, t_alpha_state_x1[nx]), axis=1)
        t_beta_state = jnp.concatenate((t_beta_state_yi, t_beta_state_x1[nx]), axis=1)
        x, t_states, c_states, prob, phase = RWKV_step(rnn_input, (last_state, t_alpha_state, t_beta_state), (last_state), num_layer, RWKV_net_params, jnp.array([ny, nx]))

        key, subkey = split(key)
        block_sample = categorical(subkey, jnp.log(prob))
        probs, phase = prob[block_sample], phase[block_sample]
        last_y2i = last_y1i
        last_y1i, t_alpha_state_yi, t_beta_state_yi= t_states
        input_yi = wemb[block_sample]
        return (input_x, input_yi, last_x1, last_x1s, last_x1e, last_x2, last_y1i, last_y2i, t_alpha_state_x1, t_beta_state_x1, t_alpha_state_yi, t_beta_state_yi, key), (last_y1i, t_alpha_state_yi, t_beta_state_yi, block_sample, probs, phase)

    def scan_fun_2d(carry_2d, indices):  # indices:[[0,0], [0,1], [0,2]...[0,Nx-1]]

        input_x, input_y, last_x1, last_x2, last_y1s, last_y1e, last_y2, t_alpha_state_x, t_beta_state_x, t_alpha_state_y, t_beta_state_y ,key = carry_2d
        ## The shape of last_y1s, last_y1e are (Ny+1) 
        index = indices[0, 0]
        print(last_y1s[index][None, ...].shape)
        print(last_x1[:-1].shape)
        last_x1s = jnp.concatenate((last_y1s[index][None, ...], last_x1[:-1]), axis=0)
        last_x1e = jnp.concatenate((last_x1[1:], last_y1e[index][None, ...]), axis=0)
        carry_1d = input_x, input_y[index], last_x1, last_x1s, last_x1e, last_x2, last_y1e[index+1], last_y2[index], t_alpha_state_x, t_beta_state_x, t_alpha_state_y[index], t_beta_state_y[index], key
        _, y = scan(scan_fun_1d, carry_1d, indices)
        '''
        The stacked y1i becomes the x1 in the next row
        The stacked y2i becomes the x2e in the next row
        '''
        last_x2 = last_x1 # x2 for the next row
        last_x1, t_alpha_state_x1, t_beta_state_x1, row_block_sample, row_prob, row_phase = y
        key = _[-1]
        last_x2 = jnp.flip(last_x2, 0)
        last_x1 = jnp.flip(last_x1, 0)
        t_alpha_state_x1 = jnp.flip(t_alpha_state_x1, 0)
        t_beta_state_x1 = jnp.flip(t_beta_state_x1, 0)
        input_x = wemb[jnp.flip(row_block_sample)]
        row_block_sample = lax.cond(index % 2, lambda x: jnp.flip(x, 0), lambda x: x, row_block_sample)
        row_prob = lax.cond(index % 2, lambda x: jnp.flip(x, 0), lambda x: x, row_prob)
        row_phase = lax.cond(index % 2, lambda x: jnp.flip(x, 0), lambda x: x, row_phase)

        return (input_x, input_y, last_x1, last_x2, last_y1s, last_y1e, last_y2, t_alpha_state_x1, t_beta_state_x1, t_alpha_state_y, t_beta_state_y, key), (row_block_sample, row_prob, row_phase)

    # initialization
    wemb = params[0]
    init = (*params[1:12], key)
    RWKV_net_params = params[12:]
    __, (samples, probs, phase) = scan(scan_fun_2d, init, ny_nx_indices)
    samples = vmap(int_to_binary, 0)(samples).reshape(Ny*py, Nx*px)
    log_probs, phase = jnp.sum(jnp.log(probs)), jnp.sum(phase)
    samples_log_amp = log_probs / 2 + phase * 1j

    return samples, samples_log_amp



In [21]:
key = PRNGKey(0)
params = init_RWKV_params(input_size=2, emb_size=8, h_size=5, num_layer=3, out_size=2, Ny=3, Nx=3, key=key)
fixed_params = (3, 3, 1, 1, 0, 0, 3)
ny_nx_indices = jnp.array([[[i, j] for j in range(3)] for i in range(3)])
key = PRNGKey(0)
sample_prob(params, fixed_params, ny_nx_indices, key)

(Array([[0, 0, 1],
        [0, 0, 1],
        [0, 1, 0]], dtype=int32),
 Array(-3.1191623+0.j, dtype=complex64))

In [20]:
from jax import nn
from jax import lax

def time_mixing(x, t_state, decay, bonus, t_mix_k, t_mix_v, t_mix_r, t_wlast_x, Wk, Wv, Wr, Wout, t_xout, t_alphaout, t_betaout):
    last_x, last_alpha, last_beta = t_state
    last_x = t_wlast_x @ last_x
    print("t:", t_mix_k.shape)
    k = Wk @ (x * t_mix_k + last_x * (1 - t_mix_k))
    v = Wv @ (x * t_mix_v + last_x * (1 - t_mix_v))
    r = Wr @ (x * t_mix_r + last_x * (1 - t_mix_r))
    wkv = (last_alpha + jnp.exp(bonus + k) * v) / \
          (last_beta + jnp.exp(bonus + k))
    rwkv = nn.sigmoid(r) * wkv

    alpha = jnp.exp(-jnp.exp(decay)) * last_alpha + jnp.exp(k) * v
    beta = jnp.exp(-jnp.exp(decay)) * last_beta + jnp.exp(k)
    alpha = t_alphaout @ alpha
    beta = t_betaout @ beta
    x = t_xout @ x
    return Wout @ rwkv, (x, alpha, beta)

def channel_mixing(x, c_states, c_mix_k, c_mix_r, c_wlast_x, Wk, Wr, Wv, c_xout):
    last_x = c_states #change tuple into array
    last_x = c_wlast_x @ last_x

    k = Wk @ (x * c_mix_k + last_x * (1 - c_mix_k))
    r = Wr @ (x * c_mix_r + last_x * (1 - c_mix_r))    
    vk = Wv @ nn.relu(k) ** 2
    x = c_xout @ x
    return nn.sigmoid(r) * vk, x

def layer_norm(x, w, b):
    mean = jnp.mean(x)
    std =  jnp.sqrt((jnp.sum((x - mean)**2) + 1e-10)/(x.size-1))
    return (x - mean)/ std * w + b

def RWKV_step(x, t_states, c_states, num_layer, RWKV_net_params, indices):
    ny, nx = indices
    w_in, b_in, whead, bhead, w_out, b_out, wprob, bprob, wphase, bphase, RWKV_cell_params = RWKV_net_params
    x = layer_norm(x, w_in[ny, nx], b_in[ny, nx])
    _ , y = lax.scan(partial(RWKV_cell, params = tuple(px[nx] for px in tuple(py[ny] for py in RWKV_cell_params)), cell_t_states = t_states, cell_c_states = c_states), x, jnp.arange(num_layer))
    x = _[0]
    t_states, c_states = y
    x = whead[ny, nx] @ layer_norm(x, w_out[ny, nx], b_out[ny, nx]) + bhead[ny, nx]
    prob = nn.softmax(wprob @ x + bprob)
    phase = 2*jnp.pi*nn.soft_sign(wphase @ x + bphase)
    return x, t_states, c_states, prob, phase

def RWKV_cell(carry, i, params, cell_t_states, cell_c_states): # carry = (x, t_states, c_states)
    #modify it for different layer of t_state and c_state and x . 
    x = carry
    wln_i, bln_i, wln_m, bln_m, decay, bonus, t_mix_r, t_mix_v, t_mix_k, t_wr, t_wv, t_wk, t_wout,t_wlast_x, c_mix_k, c_mix_r, c_wr, c_wv, c_wk, c_wlast_x, t_xout, t_alphaout, t_betaout, c_xout = tuple(p[i] for p in params) 
    layer_t_states = tuple(t[i] for t in cell_t_states)
    layer_c_states = cell_c_states[i]

    x_ = layer_norm(x, wln_i, bln_i)
    dx, output_t_states = time_mixing(x_, layer_t_states, decay, bonus, t_mix_r, t_mix_v, t_mix_k, t_wlast_x, t_wr, t_wv, t_wk, t_wout,  t_xout, t_alphaout, t_betaout)
    x = x + dx
    x_ = layer_norm(x, wln_m, bln_m)
    dx, output_c_states = channel_mixing(x_, layer_c_states, c_mix_k, c_mix_r, c_wlast_x, c_wr, c_wv, c_wk, c_xout)
    x = x + dx
    # carry need to be modified
    return x, (output_t_states, output_c_states)

In [289]:
key = PRNGKey(0)
params = init_RWKV_params(input_size=2, emb_size=5, h_size=7, num_layer=4, out_size=2, Ny=3, Nx=3, key=key)
wemb, x_init, y_init, last_x1_init, last_x2_init, last_y1s_init, last_y1e_init, last_y2_init, t_alpha_init_x, t_alpha_init_y, t_beta_init_x, t_beta_init_y,  wln_in, bln_in, wln_out, bln_out, whead, bhead, wprob, bprob, wphase, bphase, RWKV_cell_params = params
x = jnp.concatenate((wemb[0], wemb[1]), axis = 0)
last_x = jnp.concatenate((last_y2_init[0], last_y1s_init[0], last_x2_init[0], last_x1_init[1]), axis = 1)
t_alpha_init = jnp.concatenate((t_alpha_init_x[0], t_alpha_init_y[0]), axis = 1)
t_beta_init = jnp.concatenate((t_beta_init_x[0], t_beta_init_y[0]), axis = 1)


In [265]:
wln_i, bln_i, wln_m, bln_m, decay, bonus, t_mix_r, t_mix_v, t_mix_k, t_wr, t_wv, t_wk, t_wout,t_wlast_x, c_mix_k, c_mix_r, c_wr, c_wv, c_wk, c_wlast_x = RWKV_cell_params
for params in RWKV_cell_params:
    print(params.shape)

(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 14)
(3, 3, 4, 14)
(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 14, 10)
(3, 3, 4, 14, 10)
(3, 3, 4, 14, 10)
(3, 3, 4, 10, 14)
(3, 3, 4, 10, 20)
(3, 3, 4, 10)
(3, 3, 4, 10)
(3, 3, 4, 10, 10)
(3, 3, 4, 10, 10)
(3, 3, 4, 10, 10)
(3, 3, 4, 10, 20)


In [266]:
fixed_params = (3, 3, 1, 1, 0, 0, 4)
ny_nx_indices = jnp.array([[[i, j] for j in range(3)] for i in range(3)])
key = PRNGKey(0)
sample_prob(params, fixed_params, ny_nx_indices, key)

(3, 4, 10, 20)
(3, 4, 10, 20)
(2,)
(3, 4, 10, 20)
(3, 4, 10, 20)
(2,)


ValueError: not enough values to unpack (expected 12, got 3)

In [4]:
import jax
import jax.numpy as jnp
h_size = 6
num_layer = 2 
(-5 + jnp.array([8*(jnp.arange(h_size)/(h_size-1))**(0.7 + 1.3*i/(num_layer-1)) for i in range(num_layer)]))

Array([[-5.        , -2.4069493 , -0.7875767 ,  0.5949459 ,  1.8431015 ,
         3.        ],
       [-5.        , -4.68      , -3.7199998 , -2.12      ,  0.12000036,
         3.        ]], dtype=float32)

In [3]:
0.5*(jnp.arange(h_size)%3-1)+jnp.log(0.3)

Array([-1.7039728, -1.2039728, -0.7039728, -1.7039728, -1.2039728,
       -0.7039728], dtype=float32, weak_type=True)

In [5]:
a = jnp.array([1,2,3,4])
b = a[:2]
b = jnp.array([1, 2])

In [6]:
a[:2] = jnp.array([1, 2])

TypeError: '<class 'jaxlib.xla_extension.ArrayImpl'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html